# Just info about your GPU and it's memory if you use collab

In [1]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil > /dev/null
!pip install psutil > /dev/null
!pip install humanize > /dev/null
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guara|nteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

Gen RAM Free: 12.7 GB  | Proc size: 159.4 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total 15079MB


# Load our github repo

In [2]:
!git clone https://github.com/Vitaly-Protasov/DL_project_skoltech

Cloning into 'DL_project_skoltech'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 632 (delta 26), reused 0 (delta 0), pack-reused 590
Receiving objects: 100% (632/632), 141.43 KiB | 842.00 KiB/s, done.
Resolving deltas: 100% (395/395), done.


In [3]:
cd DL_project_skoltech/

/content/DL_project_skoltech


# Download data script

In [4]:
!bash download_data.sh

--2020-06-07 10:00:21--  https://s3.amazonaws.com/code2vec/data/java-small_data.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.15.54
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.15.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 382731969 (365M) [application/x-tar]
Saving to: ‘java-small_data.tar.gz’

java-small_data.tar 100%[===================>] 365.00M  32.1MB/s    in 12s     

2020-06-07 10:00:34 (30.2 MB/s) - ‘java-small_data.tar.gz’ saved [382731969/382731969]

java-small/
java-small/java-small.dict.c2v
java-small/java-small.train.c2v
java-small/java-small.val.c2v
java-small/java-small.test.c2v


# Install transformer lib

In [ ]:
! pip install transformers > /dev/null

# Import our modules and torch


In [1]:
import create_vocab
import data_to_tensors
import model_implementation
from train_class import TrainingModule

import torch
import torch.nn as nn
import torch.nn.functional as F
import random 

from torch.utils.data import DataLoader

# Because their data looks like this: we have 3 main dictionaries:
* dictionary of all words in the sheets
* dictionary of all paths(they are already hashed and converted to digits)
* dictionary of targets(method names)


In [2]:
dict_path = 'data/unique_ids2_emb/unique_ids2_emb.dict.c2v'
word2idx, path2idx, target2idx, idx2target = create_vocab.create_vocab(dict_path)

* 1) data_iterator Returns 1024 batches each(the size can be changed when calling the function)
* 2) from below you can see how the data is accessed, then 3 vectors are fed to the model(sorry for such names, I think you will understand from the arguments of the model that the output is)
* 3) well, at the output we have code_vector and the second vector-probability distribution between all labels in target_vocab(dictionary of method names)

In [3]:
path_for_train = 'data/unique_ids2_emb/unique_ids2_emb.train.c2v'
train_dataset = data_to_tensors.TextDataset(path_for_train, 
                                                    word2idx, 
                                                    path2idx, 
                                                    target2idx)

path_for_val = 'data/unique_ids2_emb/unique_ids2_emb.val.c2v'
val_dataset = data_to_tensors.TextDataset(path_for_val, 
                                                    word2idx, 
                                                    path2idx, 
                                                    target2idx)

path_for_test = 'data/unique_ids2_emb/unique_ids2_emb.test.c2v'
test_dataset = data_to_tensors.TextDataset(path_for_test, 
                                                    word2idx, 
                                                    path2idx, 
                                                    target2idx)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

# Train

In [4]:
SEED = 1337
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Model choose

* if you want to use the original model: bert = False
* if you want to use Bert model: bert = True


In [5]:
bert = True

In [6]:
# In case of bert
bert_params = dict()
bert_params['num_attention_heads'] = 1
bert_params['num_transformer_layers'] = 1
bert_params['intermediate_size'] = 32

if bert:
    model = model_implementation.code2vec_model(values_vocab_size = len(word2idx), 
                             paths_vocab_size = len(path2idx), 
                             labels_num = len(target2idx), bert=bert, bert_params=bert_params)

In [7]:
# In case if not bert
if not bert:
    model = model_implementation.code2vec_model(values_vocab_size = len(word2idx), 
                                 paths_vocab_size = len(path2idx), 
                                 labels_num = len(target2idx))

In [8]:
N_EPOCHS = 20
LR = 3e-3
WD = 0.8e-5

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WD)
criterion = nn.CrossEntropyLoss()

train_class = TrainingModule(model, optimizer, criterion, train_loader, val_loader, test_loader, N_EPOCHS, idx2target)
list_train_loss, list_val_loss, list_train_precision, list_val_precision,list_train_recall, list_val_recall, list_train_f1, list_val_f1 = train_class.train()


Epoch 1: train loss - 2.92307, validation loss - 2.57806
	 Validation: precision - 0.5, recall - 0.54545, f1_score - 0.52174
Elapsed time: 1.475
----------------------------------------------------------------------
Epoch 2: train loss - 2.49902, validation loss - 2.05433
	 Validation: precision - 0.72727, recall - 0.72727, f1_score - 0.72727
Elapsed time: 1.088
----------------------------------------------------------------------
Epoch 3: train loss - 1.92294, validation loss - 1.46456
	 Validation: precision - 0.81818, recall - 0.81818, f1_score - 0.81818
Elapsed time: 1.031
----------------------------------------------------------------------
Epoch 4: train loss - 1.12418, validation loss - 1.21098
	 Validation: precision - 0.81818, recall - 0.81818, f1_score - 0.81818
Elapsed time: 1.110
----------------------------------------------------------------------
Epoch 5: train loss - 0.62579, validation loss - 1.27102
	 Validation: precision - 0.81818, recall - 0.81818, f1_score - 0.8

# Predict and show names!

Go to weights of our models in order to take a view on how they pridect names: https://drive.google.com/drive/u/2/folders/1Q5ixv8dQ_qYqHg6w4Ep_XNeCJYZE6Cl2

You need to:
* Visit the above Drive folder.
* Right click on folder name -> "Add a shortcut to Drive" -> select "My Drive" -> "Add shortcut".

### Have an access to these weights

In [9]:
# Detect if we are in Google Colaboratory
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

from pathlib import Path
# Determine the locations of auxiliary libraries and datasets.
# `AUX_DATA_ROOT` is where 'notmnist.py', 'animation.py' and 'tiny-imagenet-2020.zip' are.
if IN_COLAB:
    google.colab.drive.mount("/content/drive/")
    
    # Change this if you created the shortcut in a different location
    AUX_DATA_ROOT = Path("/content/drive/My Drive/best_models/")
    
    assert AUX_DATA_ROOT.is_dir(), "Have you forgot to 'Add a shortcut to Drive'?"
else:
    AUX_DATA_ROOT = Path(".")

#### DEFINE model
* If you want to test original paper model:

In [10]:
if bert == False:
  state_dict = torch.load('best_model.pth')
else:
  state_dict = torch.load('best_model.pth')

model.load_state_dict(state_dict)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = model.to(DEVICE)

#### Let's test it on the test dataset

#  \<UNK\> means that this name doen't exist in our dictionary of labels:(

In [11]:
import pandas as pd
from IPython.display import display

d = {'Original names': [], 'Predicted names': []}

for start, path, end, label in iter(test_loader):
    # get from model
    code, y_pred = model(start.to(DEVICE), path.to(DEVICE), end.to(DEVICE))
    # get probability
    y_pred = F.softmax(y_pred)
    # get best name index
    y_pred = torch.argmax(y_pred, dim = 1)
    
    for i, j in zip(label, y_pred):
      d['Original names'].append(idx2target[i.item()])
      d['Predicted names'].append(idx2target[j.item()])
    break

df = pd.DataFrame(data=d)
display(df,)

/Users/unaissiddiqui/opt/anaconda3/envs/Fyp/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


,Original names,Predicted names
0,<unk>,functiondecl|sumission|num
1,main,main
2,main,main
3,main,main
4,main,main
5,main,main
6,main,main
7,<unk>,main
8,<unk>,main
9,main,main
